# 1. 뱀게임 기초 구현

이 노트북에서는 뱀게임의 기본적인 구현과 게임 환경을 이해해보겠습니다.

## 목차
1. [뱀게임 규칙과 구조](#1-뱀게임-규칙과-구조)
2. [기본 뱀게임 구현](#2-기본-뱀게임-구현)
3. [상태 표현과 행동 정의](#3-상태-표현과-행동-정의)
4. [보상 시스템 설계](#4-보상-시스템-설계)
5. [인간 플레이어와 AI 비교](#5-인간-플레이어와-ai-비교)

---


## 1. 뱀게임 규칙과 구조

### 게임 규칙
- 뱀은 격자 위에서 이동합니다
- 음식을 먹으면 뱀의 길이가 늘어납니다
- 벽이나 자신의 몸에 부딪히면 게임이 끝납니다
- 목표: 최대한 많은 음식을 먹고 오래 생존하기

### 게임 구성 요소
- **뱀 (Snake)**: 머리와 몸통으로 구성
- **음식 (Food)**: 무작위 위치에 생성
- **격자 (Grid)**: 게임이 진행되는 공간
- **점수 (Score)**: 먹은 음식의 개수


In [ ]:
# 필요한 라이브러리 import
import numpy as np
import matplotlib.pyplot as plt
import random
from collections import deque
import time

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

print("라이브러리 로드 완료!")


## 2. 기본 뱀게임 구현

간단한 뱀게임을 단계별로 구현해보겠습니다.


In [ ]:
class SimpleSnakeGame:
    """간단한 뱀게임 구현"""
    
    def __init__(self, width=10, height=10):
        self.width = width
        self.height = height
        self.reset()
    
    def reset(self):
        """게임 초기화"""
        # 뱀 초기 위치 (중앙)
        self.snake = [(self.width//2, self.height//2)]
        self.direction = (0, 1)  # 오른쪽으로 시작
        
        # 음식 위치
        self.place_food()
        
        # 게임 상태
        self.score = 0
        self.steps = 0
        self.done = False
        
        return self.get_state()
    
    def place_food(self):
        """음식을 무작위 위치에 배치"""
        while True:
            food_pos = (random.randint(0, self.width-1), 
                       random.randint(0, self.height-1))
            if food_pos not in self.snake:
                self.food = food_pos
                break
    
    def step(self, action):
        """한 스텝 진행"""
        if self.done:
            return self.get_state(), 0, self.done
        
        # 행동에 따른 방향 변경
        directions = [(0, -1), (1, 0), (0, 1), (-1, 0)]  # 상, 우, 하, 좌
        if action < len(directions):
            new_direction = directions[action]
            # 반대 방향으로 갈 수 없음
            if new_direction != (-self.direction[0], -self.direction[1]):
                self.direction = new_direction
        
        # 새로운 머리 위치
        head_x, head_y = self.snake[0]
        new_head = (head_x + self.direction[0], head_y + self.direction[1])
        
        # 충돌 검사
        if (new_head[0] < 0 or new_head[0] >= self.width or
            new_head[1] < 0 or new_head[1] >= self.height or
            new_head in self.snake):
            self.done = True
            return self.get_state(), -10, self.done  # 충돌 시 음의 보상
        
        # 뱀 이동
        self.snake.insert(0, new_head)
        
        # 음식 먹기
        if new_head == self.food:
            self.score += 1
            self.place_food()
            reward = 10  # 음식 먹을 때 양의 보상
        else:
            self.snake.pop()  # 꼬리 제거
            reward = 0.1  # 생존 보상
        
        self.steps += 1
        
        # 너무 오래 움직이면 게임 종료
        if self.steps > 200:
            self.done = True
        
        return self.get_state(), reward, self.done
    
    def get_state(self):
        """현재 상태 반환"""
        return {
            'snake': self.snake.copy(),
            'food': self.food,
            'score': self.score,
            'steps': self.steps,
            'done': self.done
        }
    
    def render(self):
        """게임 상태 시각화"""
        grid = np.zeros((self.height, self.width))
        
        # 뱀 그리기 (머리: 2, 몸통: 1)
        for i, (x, y) in enumerate(self.snake):
            if i == 0:
                grid[y, x] = 2  # 머리
            else:
                grid[y, x] = 1  # 몸통
        
        # 음식 그리기 (3)
        grid[self.food[1], self.food[0]] = 3
        
        # 시각화
        plt.figure(figsize=(8, 8))
        plt.imshow(grid, cmap='viridis')
        plt.title(f'Score: {self.score}, Steps: {self.steps}')
        plt.colorbar(label='뱀머리(2), 뱀몸(1), 음식(3)')
        plt.show()

# 게임 테스트
game = SimpleSnakeGame()
print("뱀게임 초기 상태:")
game.render()


In [ ]:
# 몇 번의 랜덤 액션으로 게임 진행 테스트
print("랜덤 액션으로 게임 진행:")

game.reset()
for i in range(10):
    action = random.randint(0, 3)  # 0:상, 1:우, 2:하, 3:좌
    state, reward, done = game.step(action)
    print(f"Step {i+1}: Action={action}, Reward={reward:.1f}, Score={state['score']}")
    
    if done:
        print("게임 종료!")
        break

print(f"최종 점수: {game.score}")
